In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/basketball/nba.sqlite
/kaggle/input/basketball/csv/other_stats.csv
/kaggle/input/basketball/csv/draft_history.csv
/kaggle/input/basketball/csv/team_details.csv
/kaggle/input/basketball/csv/player.csv
/kaggle/input/basketball/csv/draft_combine_stats.csv
/kaggle/input/basketball/csv/team_info_common.csv
/kaggle/input/basketball/csv/common_player_info.csv
/kaggle/input/basketball/csv/team_history.csv
/kaggle/input/basketball/csv/game.csv
/kaggle/input/basketball/csv/game_summary.csv
/kaggle/input/basketball/csv/officials.csv
/kaggle/input/basketball/csv/game_info.csv
/kaggle/input/basketball/csv/line_score.csv
/kaggle/input/basketball/csv/play_by_play.csv
/kaggle/input/basketball/csv/team.csv
/kaggle/input/basketball/csv/inactive_players.csv


In [2]:
import sqlite3 as sql
import plotly.express as px
import os

In [3]:
db_path = '/kaggle/input/basketball/nba.sqlite'
connection = sql.connect(db_path)
table = pd.read_sql_query("SELECT name FROM sqlite_master WHERE type = 'table'", connection)
print(table)

                   name
0                  game
1          game_summary
2           other_stats
3             officials
4      inactive_players
5             game_info
6            line_score
7          play_by_play
8                player
9                  team
10   common_player_info
11         team_details
12         team_history
13  draft_combine_stats
14        draft_history
15     team_info_common


#### How does the NBA drafting count change over time from 1949 to 2020?

In [4]:
cursor = connection.execute('select * from draft_history')
names = list(map(lambda x: x[0], cursor.description))
print('Coulmn Names Are :')
print(names)
print()
draft = pd.read_sql_query("SELECT * FROM draft_history", connection)
draft

Coulmn Names Are :
['person_id', 'player_name', 'season', 'round_number', 'round_pick', 'overall_pick', 'draft_type', 'team_id', 'team_city', 'team_name', 'team_abbreviation', 'organization', 'organization_type', 'player_profile_flag']



,person_id,player_name,season,round_number,round_pick,overall_pick,draft_type,team_id,team_city,team_name,team_abbreviation,organization,organization_type,player_profile_flag
0,79299,Clifton McNeeley,1947,1,1,1,Draft,1610610031,Pittsburgh,Ironmen,PIT,Texas-El Paso,College/University,0
1,78109,Glen Selbo,1947,1,2,2,Draft,1610610035,Toronto,Huskies,HUS,Wisconsin,College/University,1
2,76649,Eddie Ehlers,1947,1,3,3,Draft,1610612738,Boston,Celtics,BOS,Purdue,College/University,1
3,79302,Walt Dropo,1947,1,4,4,Draft,1610610032,Providence,Steamrollers,PRO,Connecticut,College/University,0
4,77048,Dick Holub,1947,1,5,5,Draft,1610612752,New York,Knicks,NYK,Long Island-Brooklyn,College/University,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7201,1631161,Yannick Nzosa,2022,2,24,54,Draft,1610612764,Washington,Wizards,WAS,Unicaja Baloncesto Malaga (Spain),Other Team/Club,0
7202,1630611,Gui Santos,2022,2,25,55,Draft,1610612744,Golden State,Warriors,GSW,Associacao Atletica Ponte Preta (Brazil),Other Team/Club,0
7203,1631247,Luke Travers,2022,2,26,56,Draft,1610612739,Cleveland,Cavaliers,CLE,Perth Wildcats (Australia),Other Team/Club,0
7204,1631133,Jabari Walker,2022,2,27,57,Draft,1610612757,Portland,Trail Blazers,POR,Colorado,College/University,1


In [5]:
query = """ 
        SELECT 
            season as year_drafted,
            COUNT(DISTINCT person_id) AS total_drafted
        FROM  draft_history
        GROUP BY season
        """
year_draft_total = pd.read_sql(query, connection).astype({"year_drafted": int})
year_draft_total

,year_drafted,total_drafted
0,1947,78
1,1948,113
2,1949,75
3,1950,121
4,1952,106
...,...,...
62,2018,60
63,2019,60
64,2020,60
65,2021,60


In [6]:
px.line(year_draft_total, x = 'year_drafted', y = 'total_drafted',
       title = 'NBA Drafting Trend(1949 to 2020)')

####  What are the top 10 NBA team that drafted most number of player from university?

In [7]:
# organization_type , team_name
query = """
        SELECT team_name,
        COUNT(DISTINCT person_id) as value_count
        FROM draft_history
        GROUP BY team_name
        HAVING organization_type = 'College/University'
        ORDER BY value_count DESC
        LIMIT 10
        """
team_count = pd.read_sql(query, connection)
team_count

,team_name,value_count
0,Hawks,430
1,Knicks,428
2,Warriors,409
3,Celtics,406
4,Lakers,386
5,Pistons,378
6,Bullets,375
7,76ers,327
8,Bulls,299
9,Suns,278


In [8]:
px.bar(team_count, x = 'team_name', y = 'value_count',
      title = 'Top 10 NBA team that drafted most number of player from university')

#### When did the teams first start to draft players from the universities?

In [9]:
# season , organization_type , team_name
query = """
        SELECT DISTINCT team_name, season AS year, organization_type
        FROM draft_history
        WHERE organization_type = 'College/University'
        ORDER BY season ASC
        """
year_draft = pd.read_sql(query, connection)
year_draft

,team_name,year,organization_type
0,Ironmen,1947,College/University
1,Huskies,1947,College/University
2,Celtics,1947,College/University
3,Steamrollers,1947,College/University
4,Knicks,1947,College/University
...,...,...,...
1344,Lakers,2022,College/University
1345,Pelicans,2022,College/University
1346,Knicks,2022,College/University
1347,Clippers,2022,College/University


#### Where are the players coming from? Do most of the players coming from high school, university, or from other professional basketball team?

In [10]:
#  organization_type , person_id
query = """
        SELECT COUNT(DISTINCT person_id) AS total_player, organization_type 
        FROM draft_history
        GROUP BY organization_type
        ORDER BY total_player DESC
        """
player_table = pd.read_sql(query, connection)
player_table

,total_player,organization_type
0,6744,College/University
1,299,Other Team/Club
2,43,High School
3,19,


In [11]:
px.pie(player_table, values = 'total_player', names = 'organization_type', hole = 0.25,
      title = 'Player Breakdown From Organization_type')

#### From 1949 to 2020, how did the total number of participated team count changes and how did the total game count changes along with it overall?

In [12]:
cursor = connection.execute('select * from game')
names = list(map(lambda x: x[0], cursor.description))
print('Coulmn Names Are :')
print(names)
print()
draft = pd.read_sql_query("SELECT * FROM game", connection)
draft

Coulmn Names Are :
['season_id', 'team_id_home', 'team_abbreviation_home', 'team_name_home', 'game_id', 'game_date', 'matchup_home', 'wl_home', 'min', 'fgm_home', 'fga_home', 'fg_pct_home', 'fg3m_home', 'fg3a_home', 'fg3_pct_home', 'ftm_home', 'fta_home', 'ft_pct_home', 'oreb_home', 'dreb_home', 'reb_home', 'ast_home', 'stl_home', 'blk_home', 'tov_home', 'pf_home', 'pts_home', 'plus_minus_home', 'video_available_home', 'team_id_away', 'team_abbreviation_away', 'team_name_away', 'matchup_away', 'wl_away', 'fgm_away', 'fga_away', 'fg_pct_away', 'fg3m_away', 'fg3a_away', 'fg3_pct_away', 'ftm_away', 'fta_away', 'ft_pct_away', 'oreb_away', 'dreb_away', 'reb_away', 'ast_away', 'stl_away', 'blk_away', 'tov_away', 'pf_away', 'pts_away', 'plus_minus_away', 'video_available_away']



,season_id,team_id_home,team_abbreviation_home,team_name_home,game_id,game_date,matchup_home,wl_home,min,fgm_home,...,dreb_away,reb_away,ast_away,stl_away,blk_away,tov_away,pf_away,pts_away,plus_minus_away,video_available_away
0,21946,1610610035,HUS,Toronto Huskies,0024600001,1946-11-01 00:00:00,HUS vs. NYK,L,0,25.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68,2,0
1,21946,1610610034,BOM,St. Louis Bombers,0024600003,1946-11-02 00:00:00,BOM vs. PIT,W,0,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,25.0,51,-5,0
2,21946,1610610032,PRO,Providence Steamrollers,0024600002,1946-11-02 00:00:00,PRO vs. BOS,W,0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53,-6,0
3,21946,1610610025,CHS,Chicago Stags,0024600004,1946-11-02 00:00:00,CHS vs. NYK,W,0,21.0,...,NaN,NaN,NaN,NaN,NaN,NaN,22.0,47,-16,0
4,21946,1610610028,DEF,Detroit Falcons,0024600005,1946-11-02 00:00:00,DEF vs. WAS,L,0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50,17,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62362,22022,1610612755,PHI,Philadelphia 76ers,0022201016,2023-03-12 00:00:00,PHI vs. WAS,W,240,41.0,...,29.0,35.0,19.0,4.0,5.0,10.0,20.0,93,-19,1
62363,22022,1610612759,SAS,San Antonio Spurs,0022201018,2023-03-12 00:00:00,SAS vs. OKC,L,240,33.0,...,43.0,55.0,26.0,8.0,3.0,9.0,19.0,102,12,1
62364,22022,1610612740,NOP,New Orleans Pelicans,0022201017,2023-03-12 00:00:00,NOP vs. POR,W,240,43.0,...,36.0,46.0,26.0,10.0,2.0,19.0,21.0,110,-17,1
62365,22022,1610612747,LAL,Los Angeles Lakers,0022201019,2023-03-12 00:00:00,LAL vs. NYK,L,240,43.0,...,34.0,46.0,18.0,8.0,3.0,11.0,18.0,112,4,1


In [13]:
query = """
    SELECT 
        season_id - 20000 AS season,
        COUNT(DISTINCT team_id_home) AS total_team_count,
        COUNT(DISTINCT game_id) AS total_game_count
    FROM game
    WHERE season <= 2020
    GROUP BY season_id
"""
year_table = pd.read_sql(query, connection)
year_table

,season,total_team_count,total_game_count
0,1946,11,331
1,1947,8,194
2,1948,12,360
3,1949,17,563
4,1950,11,354
...,...,...,...
68,2015,30,1230
69,2016,30,1230
70,2018,30,1230
71,2019,30,1059


In [14]:
from plotly.subplots import make_subplots

In [15]:
two_y_axis_plot = make_subplots(specs=[[{"secondary_y": True}]])
team_count_trace = px.bar(year_table, x="season", y="total_team_count")
game_count_trace = px.line(year_table, x="season", y="total_game_count")
team_count_trace.update_traces(name="team totals", showlegend = True, opacity=0.6)
game_count_trace.update_traces(name="game totals", showlegend = True, line_color="red")
two_y_axis_plot.add_trace(team_count_trace.data[0], secondary_y=False)
two_y_axis_plot.add_trace(game_count_trace.data[0], secondary_y=True)
two_y_axis_plot.update_yaxes(title_text="team totals", secondary_y=False)
two_y_axis_plot.update_yaxes(title_text="game totals", secondary_y=True)
two_y_axis_plot.update_layout(title_text="1946 to 2020: NBA total team and total game play trend (click legend text to filter)")

####  From 1949 to 2020, how did home game game won percentage among all the games change over time?

In [16]:
query = """
    SELECT 
        season_id - 20000 AS season, COUNT(DISTINCT game_id) AS total_game_count,
        SUM(CASE wl_home
                    WHEN 'W' THEN 1
                    ELSE 0
                    END) AS win_count
    FROM game
    GROUP BY season
"""
game_table = pd.read_sql(query, connection)
game_table['win_percentage'] = round(100 * game_table['win_count'] / game_table['total_game_count'], 2)
game_table

,season,total_game_count,win_count,win_percentage
0,1946,331,204,61.63
1,1947,194,111,57.22
2,1948,360,228,63.33
3,1949,563,392,69.63
4,1950,354,265,74.86
...,...,...,...,...
70,2018,1230,729,59.27
71,2019,1059,584,55.15
72,2020,1080,587,54.35
73,2021,1230,669,54.39


In [17]:
px.line(game_table, x = 'season', y = 'win_percentage',
       title = 'Yearly Winning Percentage of Home Game')

#### How does the free throw percentage (FT%) changes over time from 1949 to 2020? For each season, which team had the best FT%?

In [18]:
query = """
    SELECT 
        season_id - 20000 AS season,
        team_id_home AS team_id,
        team_name_home AS team_name,
        ft_pct_home AS free_throw_percentage,
        'home' AS game_location
    FROM game
    GROUP BY season_id, team_id_home
    
    UNION
    
    SELECT 
        season_id - 20000 AS season,
        team_id_away AS team_id,
        team_name_home AS team_name,
        ft_pct_away AS free_throw_percentage,
        'away' AS game_location
    FROM game
    GROUP BY season_id, team_id_away   
"""
ftp_table = pd.read_sql(query, connection)
ftp_table

,season,team_id,team_name,free_throw_percentage,game_location
0,1946,1610610025,Boston Celtics,NaN,away
1,1946,1610610025,Chicago Stags,NaN,home
2,1946,1610610026,Cleveland Rebels,NaN,home
3,1946,1610610026,St. Louis Bombers,0.667,away
4,1946,1610610028,Detroit Falcons,NaN,home
...,...,...,...,...,...
3151,2022,1610612764,Washington Wizards,0.909,home
3152,2022,1610612765,Detroit Pistons,0.792,home
3153,2022,1610612765,New York Knicks,0.594,away
3154,2022,1610612766,Charlotte Hornets,0.786,home


In [19]:
# dropping null values

ftp_table.dropna(subset = ['free_throw_percentage'], axis = 0, inplace = True)
ftp_table = ftp_table[['free_throw_percentage', 'game_location', 'season']]
ftp_table = ftp_table.groupby(['season', 'game_location']).median().reset_index()
ftp_table

,season,game_location,free_throw_percentage
0,1946,away,0.6785
1,1946,home,0.5965
2,1947,away,0.7080
3,1947,home,0.6870
4,1948,away,0.6580
...,...,...,...
139,2020,home,0.7445
140,2021,away,0.7805
141,2021,home,0.7715
142,2022,away,0.7740


In [20]:
px.line(ftp_table.query("season > 1955"), x = 'season', y = 'free_throw_percentage', color = 'game_location',
       title = 'Free Throw Percentage Over Time')

In [21]:
for i in table.name:
    query = 'select * from' + " " + i
    cursor = connection.execute(query)
    names = list(map(lambda x: x[0], cursor.description))
    print(i, 'Coulmn Names Are :')
    names.sort()
    print(names)
    print()

game Coulmn Names Are :
['ast_away', 'ast_home', 'blk_away', 'blk_home', 'dreb_away', 'dreb_home', 'fg3_pct_away', 'fg3_pct_home', 'fg3a_away', 'fg3a_home', 'fg3m_away', 'fg3m_home', 'fg_pct_away', 'fg_pct_home', 'fga_away', 'fga_home', 'fgm_away', 'fgm_home', 'ft_pct_away', 'ft_pct_home', 'fta_away', 'fta_home', 'ftm_away', 'ftm_home', 'game_date', 'game_id', 'matchup_away', 'matchup_home', 'min', 'oreb_away', 'oreb_home', 'pf_away', 'pf_home', 'plus_minus_away', 'plus_minus_home', 'pts_away', 'pts_home', 'reb_away', 'reb_home', 'season_id', 'stl_away', 'stl_home', 'team_abbreviation_away', 'team_abbreviation_home', 'team_id_away', 'team_id_home', 'team_name_away', 'team_name_home', 'tov_away', 'tov_home', 'video_available_away', 'video_available_home', 'wl_away', 'wl_home']

game_summary Coulmn Names Are :
['game_date_est', 'game_id', 'game_sequence', 'game_status_id', 'game_status_text', 'gamecode', 'home_team_id', 'live_pc_time', 'live_period', 'live_period_time_bcast', 'natl_tv_br

In [22]:
query = """
        SELECT 
        *
        FROM other_stats
        JOIN other_stats ON
        other_stats.team_id_home = game_summary.home_team_id
        """
stats = pd.read_sql(query, connection)
stats

DatabaseError: Execution failed on sql '
        SELECT 
        *
        FROM other_stats
        JOIN other_stats ON
        other_stats.team_id_home = game_summary.home_team_id
        ': ambiguous column name: main.other_stats.league_id